In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from training_utils import *
import torch
import e3nn
import h5py
import numpy as np

torch.set_default_dtype(torch.float64)

In [47]:
model_kwargs = {
    'network': 'GatedConvParityNetwork', 
    'conv': 'Convolution',
    'Rs_in': [(3, 0, 1)], 'Rs_out': [(1, 0, 1)],
    'mul': 10, 'lmax': 2, 'layers': 5,
    'max_radius': 2.,  # From dataset
    'number_of_basis': 10,
}

In [48]:
model = model_from_kwargs(model_kwargs)

In [49]:
# model = load_model('acetone/acetone_5000_s2_small_then_large_batch.torch', model_kwargs=model_kwargs)

In [50]:
# read perturbed data
with h5py.File("acetone/acetone-b3lyp_d3bj-631gd-gas-NMR-pcSseg_1.hdf5", "r") as h5:
    geoms_and_shieldings = np.array(h5.get("data"))
    
shielding = torch.tensor(geoms_and_shieldings[:, :, 3], dtype=torch.float64).unsqueeze(-1)
shielding_mean, shielding_std = shielding.mean(), shielding.std()
print(shielding_mean, shielding_std)

tensor(38.1470) tensor(132.1694)


In [51]:
batch_size = 5

dataset = torch.load('acetone/acetone_geo/acetone_geometric_dataset_2499.torch')
dataset_2 = torch.load('acetone/acetone_geo/acetone_geometric_dataset_4999.torch')
dataloader = tg.data.DataListLoader(dataset + dataset_2, batch_size=batch_size, shuffle=True)

batch_size_eval = 32

test_dataset = torch.load('acetone/acetone_geo/acetone_geometric_dataset_7499.torch')
test_dataloader = tg.data.DataListLoader(test_dataset, batch_size=batch_size_eval)

test_dataset_2 = torch.load('acetone/acetone_geo/acetone_geometric_dataset_12499.torch')
test_dataloader_2 = tg.data.DataListLoader(test_dataset_2, batch_size=batch_size_eval)

In [52]:
# stuff = evaluate(model, dataloader, [loss_fn_mse, loss_fn_mae], 'cuda:1', 5)
stuff = evaluate(model, test_dataloader, [loss_fn_mse, loss_fn_mae], 'cuda:1', 5)

In [53]:
print('MSE, MAE in ppm')
stuff.cpu() * torch.tensor([shielding_std ** 2, shielding_std])

MSE, MAE in ppm


tensor([17763.4309,    76.9878])

In [54]:
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

In [55]:
print(shielding_std)

tensor(132.1694)


In [74]:
for results in train(model, optimizer, dataloader, test_dataloader, device="cuda:1", scale_loss=shielding_std):
    with open('trial_save_small_fresh.torch', 'wb') as f:
        results['model_kwargs'] = model_kwargs
        torch.save(results, f)

tensor(132.1694)


KeyboardInterrupt: 

In [58]:
batch_size = 32

dataset = torch.load('acetone/acetone_geo/acetone_geometric_dataset_2499.torch')
dataset_2 = torch.load('acetone/acetone_geo/acetone_geometric_dataset_4999.torch')
dataloader = tg.data.DataListLoader(dataset + dataset_2, batch_size=batch_size, shuffle=True)

batch_size_eval = 32

test_dataset = torch.load('acetone/acetone_geo/acetone_geometric_dataset_7499.torch')
test_dataloader = tg.data.DataListLoader(test_dataset, batch_size=batch_size_eval)

test_dataset_2 = torch.load('acetone/acetone_geo/acetone_geometric_dataset_12499.torch')
test_dataloader_2 = tg.data.DataListLoader(test_dataset_2, batch_size=batch_size_eval)

In [75]:
# optimizer.state_dict()
# optimizer.load_state_dict(torch.load('trial_save_small_fresh_then_large_3.torch')['optimizer_state_dict'])

In [81]:
for results in train(model, optimizer, dataloader, test_dataloader, device="cuda:1", scale_loss=shielding_std):
    with open('trial_save_small_fresh_then_large_4.torch', 'wb') as f:
        results['model_kwargs'] = model_kwargs
        results['optimizer_state_dict'] = optimizer.state_dict()
        torch.save(results, f)

tensor(132.1694)
0
0 0.023256438494325772 0.06758774281502303
1
1 0.024299281997717435 0.10878588771268155
2
2 0.06089909779102471 0.13667444588613611
3
3 0.031067039163834582 0.11077813202038372
4
4 0.020992056336723554 0.07819750832269377
6
6 0.03315900033748473 0.09038669058420604
8
8 0.04377081040214497 0.10746706677435931
10
10 0.008875907741764251 0.07398536610897587
13
13 0.015275169531678692 0.07669201796318766
16
16 0.019881993784602547 0.09188974737110242
20
20 0.003872629902721232 0.04662992818969591
24
24 0.03303993555847925 0.13614711336262067
28
28 0.020550604682463922 0.08464845198598286
33
33 0.05209700788139679 0.1399424574293314
38
38 0.01578286659632377 0.07969935404991953
43
43 0.016564069675294026 0.08187145729249747
48
48 0.012948603669968767 0.07056699742493842
53
53 0.03197495904109956 0.0990769836184005
58
58 0.03821668914527086 0.0998475685185112
63
63 0.09624980110292763 0.16876971585625913
68
68 0.018231170827573886 0.07392675479664505
73
73 0.08650278999442

In [82]:
# saved = torch.load('trial_save.torch')
# saved = torch.load('trial_save_small.torch')
# saved = torch.load('trial_save_small_fresh_then_large.torch')
# saved = torch.load('trial_save_small_fresh_then_large_2.torch')
# saved = torch.load('trial_save_small_fresh_then_large_3.torch')
saved = torch.load('trial_save_small_fresh_then_large_4.torch')

In [85]:
saved['dynamics'][-1]

{'step': 98,
 'wall': 7250.098932480905,
 'batch': {'loss': 0.051605455525687315, 'mean_abs': 0.10825157145981973},
 'test': {'loss': tensor(0.0323, device='cuda:1'),
  'mean_abs': tensor(0.0934, device='cuda:1')},
 'train': {'loss': tensor(0.0205, device='cuda:1'),
  'mean_abs': tensor(0.0809, device='cuda:1')}}